In [1]:
import pandas as pd
import torch
import helpful
import PIL

In [2]:
df = pd.read_csv("../data/imagenette/validation_labels.csv")

In [3]:
df

,id,label,split
0,val/n02979186/n02979186_8971.JPEG,cassette player,valid
1,val/n02979186/n02979186_14550.JPEG,cassette player,valid
2,val/n02979186/n02979186_11971.JPEG,cassette player,valid
3,val/n02979186/n02979186_11550.JPEG,cassette player,valid
4,val/n02979186/n02979186_8751.JPEG,cassette player,valid
...,...,...,...
3920,val/n03425413/n03425413_17521.JPEG,gas pump,valid
3921,val/n03425413/n03425413_20711.JPEG,gas pump,valid
3922,val/n03425413/n03425413_19050.JPEG,gas pump,valid
3923,val/n03425413/n03425413_13831.JPEG,gas pump,valid


In [4]:
model, preprocess, tokenizer = helpful.load_clip()

/Users/donnybertucci/ClipQuery/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# given a single text string, return the score for the best matches for the images
with torch.no_grad(), torch.cuda.amp.autocast():
	my_text = "a picture of a radio"
	# [1, 77]
	my_text_tensor = tokenizer([my_text])

	# [1, 512]
	my_text_features = model.encode_text(my_text_tensor)

	# now encode all the images into a single gigatensor
	
	# TODO don't do these separate
	def image_path_to_PIL(df: pd.Series):
		return df.apply(lambda x: PIL.Image.open(f"../data/imagenette/{x}"))
	def preprocess_PIL(df: pd.Series):
		return df.apply(lambda x: preprocess(x).unsqueeze(0))
	
	image_col = df["id"]
	pil_col = image_path_to_PIL(image_col)
	list_of_image_tensors = preprocess_PIL(pil_col).tolist()

	# [3925, 3, 224, 224]
	image_tensor = torch.cat(list_of_image_tensors[0:2], dim=0)

	# [3925, 512]
	image_features = model.encode_image(image_tensor)
	print(image_features.detach().cpu().tolist())
	print(image_features.shape)

	# [3925, 1]
	clip_scores = image_features @ my_text_features.T


/Users/donnybertucci/ClipQuery/venv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[[0.3085854947566986, -0.19092527031898499, 0.21958880126476288, 0.11716742813587189, -0.0037107467651367188, -0.13014525175094604, -0.6234740614891052, 0.13598228991031647, 0.6468501091003418, 0.08730864524841309, -0.39296144247055054, 0.01905985176563263, -0.738804280757904, 0.23089000582695007, 0.7054218649864197, 0.25462606549263, 0.16243022680282593, 0.13050684332847595, -0.04177182912826538, -0.09144895523786545, 0.05879032611846924, 0.3644520044326782, 0.5247053503990173, -0.24006180465221405, -0.22899164259433746, -0.072074294090271, -0.03729061037302017, -0.09867023676633835, 0.38933050632476807, -0.1423812359571457, -0.17012809216976166, -0.34584325551986694, 0.6529657244682312, -0.011208683252334595, -0.10513748228549957, 0.2447788417339325, 0.014155350625514984, -0.3857279121875763, -0.3700169324874878, -0.1929217129945755, 0.6720931529998779, -0.2722024917602539, 0.46544021368026733, -0.3181791603565216, -0.06445613503456116, 0.04570688307285309, 1.032625436782837, 0.13526